<a href="https://colab.research.google.com/github/orcuncakaar/orcuncakaar/blob/main/2_Homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [27]:
df = pd.read_csv("cardekho_data.csv")

In [28]:
df

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,ritz,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,sx4,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,ciaz,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,wagon r,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,swift,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0
...,...,...,...,...,...,...,...,...,...
296,city,2016,9.50,11.60,33988,Diesel,Dealer,Manual,0
297,brio,2015,4.00,5.90,60000,Petrol,Dealer,Manual,0
298,city,2009,3.35,11.00,87934,Petrol,Dealer,Manual,0
299,city,2017,11.50,12.50,9000,Diesel,Dealer,Manual,0


In [29]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder

In [30]:
df['Fuel_Type'] = df['Fuel_Type'].astype('category').cat.codes
df['Seller_Type'] = df['Seller_Type'].astype('category').cat.codes
df['Transmission'] = df['Transmission'].astype('category').cat.codes

In [31]:
current_year = 2024
df['Car_Age'] = current_year - df['Year']
df = df.drop(columns=['Car_Name', 'Year'])

In [32]:
df

,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner,Car_Age
0,3.35,5.59,27000,2,0,1,0,10
1,4.75,9.54,43000,1,0,1,0,11
2,7.25,9.85,6900,2,0,1,0,7
3,2.85,4.15,5200,2,0,1,0,13
4,4.60,6.87,42450,1,0,1,0,10
...,...,...,...,...,...,...,...,...
296,9.50,11.60,33988,1,0,1,0,8
297,4.00,5.90,60000,2,0,1,0,9
298,3.35,11.00,87934,2,0,1,0,15
299,11.50,12.50,9000,1,0,1,0,7


In [33]:
X = df.drop('Selling_Price', axis=1)
y = df['Selling_Price']

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [35]:
print(f"Eğitim seti boyutu: {X_train.shape[0]}")
print(f"Test seti boyutu: {X_test.shape[0]}")

Eğitim seti boyutu: 240
Test seti boyutu: 61


In [36]:
regressor = DecisionTreeRegressor(random_state=42)
regressor.fit(X_train, y_train)

DecisionTreeRegressor(random_state=42)

In [37]:
y_pred = regressor.predict(X_test)

In [38]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Ortalama Karesel Hata (MSE): {mse:.2f}")
print(f"R-kare Skoru (R²): {r2:.2f}")

Ortalama Karesel Hata (MSE): 2.00
R-kare Skoru (R²): 0.91


In [39]:
from sklearn.model_selection import GridSearchCV

In [40]:
param_grid = {
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['squared_error', 'friedman_mse', 'absolute_error']
}

In [41]:
grid_search = GridSearchCV(estimator=regressor, param_grid=param_grid,
                           cv=5, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')

In [42]:
grid_search.fit(X_train, y_train)


Fitting 5 folds for each of 162 candidates, totalling 810 fits


GridSearchCV(cv=5, estimator=DecisionTreeRegressor(random_state=42), n_jobs=-1,
             param_grid={'criterion': ['squared_error', 'friedman_mse',
                                       'absolute_error'],
                         'max_depth': [None, 10, 20, 30, 40, 50],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10]},
             scoring='neg_mean_squared_error', verbose=2)

In [43]:
print("En iyi parametreler:", grid_search.best_params_)
print("En iyi skor (Negatif MSE):", grid_search.best_score_)

En iyi parametreler: {'criterion': 'friedman_mse', 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2}
En iyi skor (Negatif MSE): -3.048574166666667


In [44]:
best_grid = grid_search.best_estimator_

In [45]:
y_pred_grid = best_grid.predict(X_test)

In [46]:
mse_grid = mean_squared_error(y_test, y_pred_grid)
r2_grid = r2_score(y_test, y_pred_grid)

In [47]:
print(f"\nGridSearchCV Sonrası Ortalama Karesel Hata (MSE): {mse_grid:.2f}")
print(f"GridSearchCV Sonrası R-kare Skoru (R²): {r2_grid:.2f}")


GridSearchCV Sonrası Ortalama Karesel Hata (MSE): 2.01
GridSearchCV Sonrası R-kare Skoru (R²): 0.91


In [48]:
from sklearn.model_selection import GridSearchCV

# Denenecek parametreleri ve aralıklarını belirleyelim
param_grid = {
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['squared_error', 'friedman_mse', 'absolute_error']
}

# GridSearchCV nesnesini oluşturalım
# n_jobs=-1 parametresini kaldırarak hatayı gideriyoruz.
grid_search = GridSearchCV(estimator=regressor, param_grid=param_grid,
                           cv=5, verbose=2, scoring='neg_mean_squared_error')

# GridSearchCV'yi eğitim verisiyle çalıştıralım
grid_search.fit(X_train, y_train)

# En iyi parametreleri ve en iyi skoru yazdıralım
print("En iyi parametreler:", grid_search.best_params_)
print("En iyi skor (Negatif MSE):", grid_search.best_score_)

# En iyi modeli seçelim
best_grid = grid_search.best_estimator_

# En iyi model ile test seti üzerinde tahmin yapalım
y_pred_grid = best_grid.predict(X_test)

# Yeni modelin performansını değerlendirelim
mse_grid = mean_squared_error(y_test, y_pred_grid)
r2_grid = r2_score(y_test, y_pred_grid)

print(f"\nGridSearchCV Sonrası Ortalama Karesel Hata (MSE): {mse_grid:.2f}")
print(f"GridSearchCV Sonrası R-kare Skoru (R²): {r2_grid:.2f}")

Fitting 5 folds for each of 162 candidates, totalling 810 fits
[CV] END criterion=squared_error, max_depth=None, min_samples_leaf=1, min_samples_split=2; total time=   0.0s
[CV] END criterion=squared_error, max_depth=None, min_samples_leaf=1, min_samples_split=2; total time=   0.0s
[CV] END criterion=squared_error, max_depth=None, min_samples_leaf=1, min_samples_split=2; total time=   0.0s
[CV] END criterion=squared_error, max_depth=None, min_samples_leaf=1, min_samples_split=2; total time=   0.0s
[CV] END criterion=squared_error, max_depth=None, min_samples_leaf=1, min_samples_split=2; total time=   0.0s
[CV] END criterion=squared_error, max_depth=None, min_samples_leaf=1, min_samples_split=5; total time=   0.0s
[CV] END criterion=squared_error, max_depth=None, min_samples_leaf=1, min_samples_split=5; total time=   0.0s
[CV] END criterion=squared_error, max_depth=None, min_samples_leaf=1, min_samples_split=5; total time=   0.0s
[CV] END criterion=squared_error, max_depth=None, min_sam

In [49]:
#This code writed in Gemini 2.5 Pro for test.

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score

# --- 1. Veriyi Yükleme ve Ön İşleme ---
# Bu kısım, daha önceki adımların hepsini tek seferde yapar.
df = pd.read_csv('cardekho_data.csv')

# Kategorik verileri sayısal verilere dönüştürme
df['Fuel_Type'] = df['Fuel_Type'].astype('category').cat.codes
df['Seller_Type'] = df['Seller_Type'].astype('category').cat.codes
df['Transmission'] = df['Transmission'].astype('category').cat.codes

# 'Year' sütunundan arabanın yaşını hesaplama ve gereksiz sütunları atma
current_year = 2024
df['Car_Age'] = current_year - df['Year']
df = df.drop(columns=['Car_Name', 'Year'])

# Bağımlı ve bağımsız değişkenleri belirleme
X = df.drop('Selling_Price', axis=1)
y = df['Selling_Price']

# Veri setini eğitim ve test olarak ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# --- 2. İki Modelin Karşılaştırılması ---

# Model 1: Başlangıç (Varsayılan) Model
baseline_model = DecisionTreeRegressor(random_state=42)
baseline_model.fit(X_train, y_train)
y_pred_baseline = baseline_model.predict(X_test)
r2_baseline = r2_score(y_test, y_pred_baseline)

# Model 2: Optimize Edilmiş Model (GridSearchCV ile bulunan en iyi parametrelerle)
optimized_params = {
    'criterion': 'friedman_mse',
    'max_depth': 10,
    'min_samples_leaf': 2,
    'min_samples_split': 5
}
optimized_model = DecisionTreeRegressor(random_state=42, **optimized_params)
optimized_model.fit(X_train, y_train)
y_pred_optimized = optimized_model.predict(X_test)
r2_optimized = r2_score(y_test, y_pred_optimized)


# --- 3. Sonuçların Yazdırılması ---
print("--- MODEL PERFORMANS KARŞILAŞTIRMASI ---")
print(f"Başlangıç Modelinin R-kare (R²) Skoru\t: {r2_baseline:.2f}")
print(f"Optimize Edilmiş Modelin R-kare (R²) Skoru\t: {r2_optimized:.2f}")

--- MODEL PERFORMANS KARŞILAŞTIRMASI ---
Başlangıç Modelinin R-kare (R²) Skoru	: 0.91
Optimize Edilmiş Modelin R-kare (R²) Skoru	: 0.93
